In [1]:
url = 'http://dapds00.nci.org.au/thredds/dodsC/gb6/BRAN/BRAN_2016/OFAM/ocean_eta_t_2000_01.nc'

In [2]:
!pip install netCDF4
!pip install xarray

In [3]:
import xarray,numpy as np

In [4]:
data = xarray.open_dataset(url)

In [5]:
eta_t = data['eta_t']

In [6]:
eta_t_numpy = np.array(eta_t)

In [7]:
eta_t_numpy = np.nan_to_num(eta_t_numpy)

In [8]:
eta_t_numpy.shape

(31, 1500, 3600)

In [14]:
input_data = np.stack([eta_t_numpy[i,:150,:360] for i in range(31)],axis=2)

In [15]:
input_data.shape

(150, 360, 31)

In [13]:
input_data[:100,:100,30]

array([[ 0.       ,  0.       ,  0.       , ...,  0.       ,  0.       ,
         0.       ],
       [ 0.       ,  0.       ,  0.       , ...,  0.       ,  0.       ,
         0.       ],
       [ 0.       ,  0.       ,  0.       , ...,  0.       ,  0.       ,
         0.       ],
       ...,
       [-1.8311106, -1.8311106, -1.8311106, ..., -1.7700735, -1.7700735,
        -1.7700735],
       [-1.8005921, -1.8005921, -1.8005921, ..., -1.8005921, -1.8005921,
        -1.8005921],
       [-1.8005921, -1.8005921, -1.8005921, ..., -1.8005921, -1.8005921,
        -1.8005921]], dtype=float32)

In [30]:
input_data

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
      dtype=float32)

In [11]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Conv2D, Activation, MaxPool2D
from tensorflow.keras.layers import BatchNormalization, Flatten, Reshape, Conv2DTranspose, LeakyReLU
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam, SGD

In [22]:
lat = 150
longg = 360
time = 31

In [23]:
class CNN_Autoencoder(Model):
  def __init__(self):
    super(CNN_Autoencoder,self).__init__()

    self.encoder = tf.keras.Sequential([
                                        Input(shape=(lat,longg,time)),
                                        Conv2D(32,(3,3),strides=(2,2)),
                                        BatchNormalization(),
                                        LeakyReLU(),
                                        MaxPool2D((2,2)),
                                        Conv2D(64,(3,3),strides=(2,2)),
                                        BatchNormalization(),
                                        LeakyReLU(),
                                        MaxPool2D((2,2)),
                                        Flatten(),
                                        Dense(63488,activation='relu'),
                                        Reshape((32,64,31))
    ])    
    self.decoder = tf.keras.Sequential([
                                       Conv2DTranspose(64,(3,3),strides=(2,2)),
                                        BatchNormalization(),
                                        LeakyReLU(),
                                        Conv2DTranspose(31,(3,3),strides=(2,2)),
                                        Flatten(),
                                        Dense(lat*longg*time),
                                        Reshape((lat,longg,time)) 
    ])

  def call(self,x):
    encoded = self.encoder(x)
    decoded = self.decoder(encoded)
    return decoded

autoencoder = CNN_Autoencoder()

ResourceExhaustedError: ignored

In [21]:
 autoencoder.compile(optimizer='adam', loss='mse')
history = autoencoder.fit(input_data, input_data,
                epochs=100)


Epoch 1/100


ValueError: ignored